# Splice site prediction on C. Elegans DNA

## Setup

### Import packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import glob

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, average_precision_score, roc_auc_score
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from imblearn.ensemble import BalancedRandomForestClassifier, RUSBoostClassifier, EasyEnsembleClassifier

from squiggle import transform

import tensorflow.keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, clone_model, Model

### Load dataset

Encode the dna string as a list of floats

In [ ]:
dna_float_encoding_dict = { 'A': 0.25, 'C': 0.5,  'G': 0.75, 'T': 1.0 }

def encode_dna_string_to_floats(dna_string):
    float_encoded_dna = []
    
    for n in dna_string:
        float_encoded_dna.append(dna_float_encoding_dict[n])
    
    return np.array(float_encoded_dna)


In [ ]:
df = pd.read_csv('data/C_elegans_acc_seq.csv', names=['labels', 'sequences'])

labels_df = df['labels']
dena_sequences_df = df['sequences']

encoded_dna_sequences = np.array([encode_dna_string_to_floats(c) for c in np.array(dena_sequences_df)])
binary_labels = np.array([0 if x == -1 else 1 for x in np.array(labels_df)])

x_train, x_test, y_train, y_test = train_test_split(encoded_dna_sequences, np.array(labels_df), test_size=0.2, random_state=29)

### Dtaset analysis

In [ ]:
labels = np.array(labels_df)
fraction_of_class1_samples = np.array(labels[labels==1]).shape[0] / labels.shape[0]
print("Class 1 (spline site) contains {0}% of the data samples.".format(round(fraction_of_class1_samples, 2) * 100))
print("For each sample of class 1 there are {0} samples of class -1".format(1/fraction_of_class1_samples))

number_of_features = x_train.shape[1]
print("Each sample consists of {0} features, i.e. nucleotides".format(number_of_features))

### Class weights

In [ ]:
weight_for_0 = (1 / y_train[y_train == -1].shape[0])*x_train.shape[0]/2.0
weight_for_1 = (1 / y_train[y_train == 1].shape[0])*x_train.shape[0]/2.0

class_weight = {-1: weight_for_0, 1: weight_for_1}
class_weight_01 = {0: weight_for_0, 1: weight_for_1}

print('Weight for class -1: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

The C. Elegans data set is highly unbalanced. the non-spline sites make more than 90% of the data set. 

In [ ]:
a =  transform(dena_sequences_df[0], method='gates')
print(a)

In [ ]:
plt.subplot(3, 2, 1)
plt.scatter(a[0], a[1])


plt.figure(figsize=(14, 12), dpi= 80, facecolor='w', edgecolor='k')

In [ ]:
fig = plt.figure(1, figsize=(5.5,5.5))


# the scatter plot:
axScatter = plt.subplot(111)
axScatter.scatter(a[0], a[1])

# set axes range
plt.xlim(-4, 12)
plt.ylim(-10, 10)

plt.plot(a[0], a[1], 'r')

### Helper functions

Takes a model wrapper and evaluates its performance on the provided evaluation set, outputs the OP, PC and IoU and saves the results in a txt file if required.

In [ ]:
def evaluate_model(model_wrapper, X, Y_true, verbose=False):
    Y_pred = model_wrapper.predict(X)
    
    f1 = f1_score(Y_true, Y_pred, average="macro")
    acc = accuracy_score(Y_true, Y_pred)
    auroc = roc_auc_score(Y_true, Y_pred)
    auprc = average_precision_score(Y_true, Y_pred)
    cm = confusion_matrix(Y_true, Y_pred)
    acc_per_class = cm.diagonal() / np.sum(cm, axis=1)
    
    if verbose:
        print("\n=================================================================")
        print("\nAccuracy:", round(acc, 5))
        print("F1:      ", round(f1, 5))
        print("AUROC    ", round(auroc, 5))
        print("AUPRC    ", round(auprc, 5))
        print("Confusion matrix:")
        print(cm)
        print("Accuracy per class")
        print(acc_per_class)
        print("\n=================================================================\n")
    return (f1, acc, acc_per_class, auroc, auprc)

Runs K-Fold cross-validation and pretty-prints intermediate results.

In [ ]:
def cross_validation(model_wrapper, X, Y, folds=10, message=''):
    kf = KFold(n_splits=folds, shuffle=True)
    i = 0.
    f1_a = 0
    acc_a = 0
    acc_per_class_a = 0
    auroc_a = 0
    auprc_a = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        model_wrapper.fit(X_train, Y_train)
        f1, acc, acc_per_class, auroc, auprc = evaluate_model(model_wrapper, X_test, Y_test)
        f1_a += f1
        acc_a += acc
        acc_per_class_a += acc_per_class
        auroc_a += auroc
        auprc_a += auprc
        i += 1
    print("\n=================================================================")
    if message != '':
        print(message)
    print("\nAverage Accuracy:", round(acc_a/i, 5))
    print("Average F1:      ", round(f1_a/i, 5))
    print("Average AUPRC:      ", round(auprc_a/i, 5))
    print("Average AUROC:      ", round(auroc_a/i, 5))
    print("Average Accuracy per class")
    print(acc_per_class_a / i)
    print("\n=================================================================\n")

In [ ]:
class Sklearn_Model_Wrapper():
    def __init__(self, model):
        self.model = model
    
    def fit(self, X, Y):
        self.model.fit(X, Y)
    
    def predict(self, X):
        return self.model.predict(X)

In [ ]:
class Keras_Model_Wrapper():
    def __init__(self, model, batch_size=64, epochs=10, loss='categorical_crossentropy', class_weights={0:1, 1:1}):
        self.model = model
        self.epochs = epochs
        self.batch_size = batch_size
        self.loss = loss
        self.class_weights = class_weights
    
    def fit(self, X, Y):
        ohe = OneHotEncoder()
        one_hot_encoded_y = ohe.fit_transform(Y.reshape(-1, 1)).toarray()
        model = clone_model(self.model)
        model.compile(loss=self.loss, optimizer='adam', metrics=['accuracy'])
        model.fit(X, one_hot_encoded_y, class_weight=self.class_weights, 
                       batch_size=self.batch_size, epochs=self.epochs, verbose=0)
        self.trained_model = model
    
    def predict(self, X):
        probas = self.trained_model.predict(X)
        predictions = np.argmax(probas, axis=1)
        return np.array([-1 if x == 0 else 1 for x in predictions])

## Models

### SVM

In [ ]:
svm = Sklearn_Model_Wrapper(LinearSVC(random_state=0, tol=1e-5, max_iter=2000))

cross_validation(svm, x_train, y_train)

In [ ]:
svm = Sklearn_Model_Wrapper(LinearSVC(random_state=0, tol=1e-5, max_iter=5000, class_weight=class_weight))

cross_validation(svm, x_train, y_train)

In [ ]:
svm = Sklearn_Model_Wrapper(LinearSVC(random_state=0, tol=1e-5, max_iter=10000, class_weight=class_weight))

cross_validation(svm, x_train, y_train)

In [ ]:
svm = Sklearn_Model_Wrapper(SVC(random_state=0, tol=1e-5, max_iter=10000, class_weight=class_weight, kernel='poly'))

cross_validation(svm, x_train, y_train)

### Random Forest

In [ ]:
random_forest = Sklearn_Model_Wrapper(RandomForestClassifier(max_depth=2, random_state=0))

cross_validation(random_forest, x_train, y_train)

In [ ]:
random_forest = Sklearn_Model_Wrapper(RandomForestClassifier(max_depth=80, random_state=0, bootstrap=True))

cross_validation(random_forest, x_train, y_train)

#### Balanced Random Forest
As proposed in Breiman (2001), random forest induces each constituent tree from a bootstrap sample of thetraining data. In learning extremely imbalanced data, there is a significant probability that a bootstrap samplecontains few or even none of the minority class, resulting in a tree with poor performance for predictingthe minority class.   A na ̈ıve way of fixing this problem is to use a stratified bootstrap;  i.e.,  sample with replacement from within each class.  
This still does not solve the imbalance problem entirely.  As recentresearch shows (e.g., Ling & Li (1998),Kubat & Matwin (1997),Drummond & Holte (2003)), for the treeclassifier, artificially making class priors equal either by down-sampling the majority class or over-samplingthe minority class is usually more effective with respect to a given performance measurement, and that down-sampling seems to have an edge over over-sampling. However, down-sampling the majority class may resultin loss of information, as a large part of the majority class is not used. Random forest inspired us to ensembletrees induced from balanced down-sampled data.  
[Ref: https://statistics.berkeley.edu/sites/default/files/tech-reports/666.pdf]

##### Max deph
Compare different values for the maximal depth of the trees.

In [ ]:
print("")
for i in range(2, 50, 3):
    random_forest_max = Sklearn_Model_Wrapper(BalancedRandomForestClassifier(max_depth=i, random_state=0))
    cross_validation(random_forest_max, x_train, y_train, message="10 fold CV for max depth={0}:".format(i))

In [ ]:
random_forest_max8 = Sklearn_Model_Wrapper(BalancedRandomForestClassifier(max_depth=8, random_state=0))
cross_validation(random_forest_max8, x_train, y_train, message="max depth={0}:".format(8))

random_forest_max9 = Sklearn_Model_Wrapper(BalancedRandomForestClassifier(max_depth=9, random_state=0))
cross_validation(random_forest_max9, x_train, y_train, message="max depth={0}:".format(9))

random_forest_max10 = Sklearn_Model_Wrapper(BalancedRandomForestClassifier(max_depth=10, random_state=0))
cross_validation(random_forest_max10, x_train, y_train, message="max depth={0}:".format(10))

In [ ]:
evaluate_model(random_forest_max10, x_test, y_test, verbose=True)

In [ ]:
rusboost = RUSBoostClassifier(n_estimators=200, algorithm='SAMME.R',random_state=17)
random_forest_max8 = Sklearn_Model_Wrapper(rusboost)
cross_validation(random_forest_max8, x_train, y_train)

his algorithm is known as EasyEnsemble. The classifier is an ensemble of AdaBoost learners trained on different balanced boostrap samples. The balancing is achieved by random under-sampling.  

[Ref: https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.ensemble.EasyEnsembleClassifier.html#imblearn.ensemble.EasyEnsembleClassifier]

In [ ]:
eec_200 = EasyEnsembleClassifier(n_estimators=200, random_state=17)
eec_wrapper = Sklearn_Model_Wrapper(eec_200)
cross_validation(random_forest_max8, x_train, y_train)

In [ ]:
eec_400 = EasyEnsembleClassifier(n_estimators=400, random_state=17, n_jobs=4)
eec_wrapper = Sklearn_Model_Wrapper(eec_400)
cross_validation(eec_wrapper, x_train, y_train)

In [ ]:
eec_600 = EasyEnsembleClassifier(n_estimators=600, random_state=17, n_jobs=4)
eec_wrapper = Sklearn_Model_Wrapper(eec_600)
cross_validation(eec_wrapper, eec_wrapper, x_train, y_train)

In [ ]:
eec_600 = EasyEnsembleClassifier(n_estimators=400, random_state=17, n_jobs=4, replacement=True)
eec_wrapper = Sklearn_Model_Wrapper(eec_600)
cross_validation(eec_wrapper, x_train, y_train)

### Logistic Regression

In [ ]:
logistic_regression = Sklearn_Model_Wrapper(LogisticRegression(random_state=13, max_iter=200))
cross_validation(logistic_regression, x_train, y_train)

In [ ]:
weights = {-1:1, 1:10}

logistic_regression = LogisticRegression(random_state=13, max_iter=200, class_weight=weights)
logistic_regression_wrapper = Sklearn_Model_Wrapper(logistic_regression)
cross_validation(logistic_regression_wrapper, x_train, y_train)

In [ ]:
weights = {-1:1, 1:100}

logistic_regression = LogisticRegression(random_state=13, max_iter=200, class_weight=weights)
logistic_regression_wrapper = Sklearn_Model_Wrapper(logistic_regression)
cross_validation(logistic_regression_wrapper, x_train, y_train)

As we set the weights havely towards the minority class we can the the calassifier overfitting to that class.  
In the next experiment we set the class weights to according to the class distribution of the data set. 
Next we use the default class weights calculated at the beginning.

In [ ]:
logistic_regression = LogisticRegression(random_state=13, max_iter=200, class_weight=class_weight)
logistic_regression_wrapper = Sklearn_Model_Wrapper(logistic_regression)
cross_validation(logistic_regression_wrapper, x_train, y_train)

In the next cell we experiment with different solvers. We increase the max. interations as some solver apperently do not converge with few iterations.

In [ ]:
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']

for solver in solvers:
    logistic_regression = LogisticRegression(random_state=13, max_iter=10000, class_weight=class_weight, solver=solver)
    logistic_regression_wrapper = Sklearn_Model_Wrapper(logistic_regression)
    cross_validation(logistic_regression_wrapper, x_train, y_train, message='solver={0}'.format(solver))

Next we use L1 instead of L2 loss. Intuitively L1 seems a better fit than L2 as the features are discrete values.  
Indeed we can observe a slight increase of the F1 score and in the accuracies of both classes for the liblinear solver using L1 loss. 

In [ ]:
l1_solvers = ['liblinear', 'saga']

for solver in l1_solvers:
    logistic_regression = LogisticRegression(random_state=13, max_iter=10000, class_weight=class_weight, solver=solver, penalty='l1')
    logistic_regression_wrapper = Sklearn_Model_Wrapper(logistic_regression)
    cross_validation(logistic_regression_wrapper, x_train, y_train, message='solver={0} and loss=L1'.format(solver))

### Deep Learning
In this section we experiment with deep learning models.  
Because the data only has 82 descret features we focus on simple fully connected networks.  
Furthermore we are using five folds instead of ten for our cross-validation to remove bias from our model evaluation.  
As deep learning typically requires many samples to properly train on and the Elegants data set is rather small we need to be extra careful to prevent overfitting. As you can see in the section below, even model with only two fully connected layers have far more trainable parameters than we have data features. 

#### Fully connected networks

Our first model is a simple fully connected model with three layers. 

In [ ]:
def get_fully_connected_model():
    fully_connected_model = Sequential()
    fully_connected_model.add(Dense(40, activation='relu'))
    fully_connected_model.add(Dense(12, activation='relu'))
    fully_connected_model.add(Dense(2, activation='softmax'))

    return fully_connected_model

In [ ]:
fully_connected_model = get_fully_connected_model()
fully_connected_model.build(input_shape=x_train.shape)
fully_connected_model.summary()

In [ ]:
fully_connected_model_wrapper = Keras_Model_Wrapper(get_fully_connected_model())
cross_validation(fully_connected_model_wrapper, x_train, y_train, folds=5)

In [ ]:
fully_connected_model_wrapper1 = Keras_Model_Wrapper(get_fully_connected_model(), class_weights=class_weight_01)
cross_validation(fully_connected_model_wrapper1, x_train, y_train, folds=5)

In [ ]:
fully_connected_model_wrapper = Keras_Model_Wrapper(get_fully_connected_model(), class_weights=class_weight_01, epochs=10)
cross_validation(fully_connected_model_wrapper, x_train, y_train, folds=5)

In [ ]:
fully_connected_model_wrapper = Keras_Model_Wrapper(get_fully_connected_model(), class_weights=class_weight_01, epochs=50)
cross_validation(fully_connected_model_wrapper, x_train, y_train, folds=5)

In [ ]:
fully_connected_model_wrapper = Keras_Model_Wrapper(get_fully_connected_model(), loss='binary_crossentropy', class_weights=class_weight_01, epochs=50)
cross_validation(fully_connected_model_wrapper, x_train, y_train, folds=5)

In [ ]:
fully_connected_model_wrapper = Keras_Model_Wrapper(get_fully_connected_model(), class_weights=class_weight_01, epochs=100)
cross_validation(fully_connected_model_wrapper, x_train, y_train, folds=5)

In [ ]:
fully_connected_model_wrapper = Keras_Model_Wrapper(get_fully_connected_model(), loss='binary_crossentropy', class_weights=class_weight_01, epochs=100)
cross_validation(fully_connected_model_wrapper, x_train, y_train, folds=5)

In [ ]:
def get_fully_connected_model2():
    fully_connected_model = Sequential()
    fully_connected_model.add(Dense(100, activation='relu'))
    fully_connected_model.add(Dense(50, activation='relu'))
    fully_connected_model.add(Dense(20, activation='relu'))
    fully_connected_model.add(Dense(2, activation='softmax'))

    fully_connected_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return fully_connected_model

In [ ]:
fully_connected_model = get_fully_connected_model2()
fully_connected_model.build(input_shape=x_train.shape)
fully_connected_model.summary()

In [ ]:
fully_connected_model_wrapper = Keras_Model_Wrapper(get_fully_connected_model2(), class_weights=class_weight_01, epochs=50)
cross_validation(fully_connected_model_wrapper, x_train, y_train)

In [ ]:
fully_connected_model_wrapper = Keras_Model_Wrapper(get_fully_connected_model2(), loss='binary_crossentropy', class_weights=class_weight_01, epochs=50)
cross_validation(fully_connected_model_wrapper, x_train, y_train)

In [ ]:
fully_connected_model_wrapper = Keras_Model_Wrapper(get_fully_connected_model2(), class_weights=class_weight_01, epochs=100)
cross_validation(fully_connected_model_wrapper, x_train, y_train)

In [ ]:
def get_fully_connected_model3():
    fully_connected_model = Sequential()
    fully_connected_model.add(Dense(2, activation='relu'))
    fully_connected_model.add(Dense(2, activation='softmax'))

    return fully_connected_model

In [ ]:
fully_connected_model = get_fully_connected_model3()
fully_connected_model.build(input_shape=x_train.shape)
fully_connected_model.summary()

In [ ]:
fully_connected_model_wrapper = Keras_Model_Wrapper(get_fully_connected_model2(), class_weights=class_weight_01, epochs=50)
cross_validation(fully_connected_model_wrapper, x_train, y_train)

#### LSTM

In [254]:
def lstm_attention():
    X = Input(shape=(82,1))
    rnn = Bidirectional(LSTM(128, return_sequences=True, input_shape=(82, 1)))(X)

    attentions = []
    for _ in range(4):
        Q = Dense(8, use_bias=False)(rnn)
        V = Dense(8, use_bias=False)(rnn)
        K = Dense(8, use_bias=False)(rnn)
        attentions.append(Attention()([Q, V, K]))
    c = Concatenate()(attentions)
    b = BatchNormalization()(c)
    f = Flatten()(b)
    d = Dropout(.2)(f)
    d = Dense(128, activation='relu')(d)
    d = Dense(128, activation='relu')(d)
    y = Dense(2, activation='softmax', bias_initializer=None)(d)

    m = Model(X, y)
    return m

In [255]:
lstm = lstm_attention()
lstm.build(input_shape=x_train.shape)
lstm.summary()

Model: "model_37"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_50 (InputLayer)           [(None, 82, 1)]      0                                            
__________________________________________________________________________________________________
bidirectional_38 (Bidirectional (None, 82, 256)      133120      input_50[0][0]                   
__________________________________________________________________________________________________
dense_522 (Dense)               (None, 82, 8)        2048        bidirectional_38[0][0]           
__________________________________________________________________________________________________
dense_523 (Dense)               (None, 82, 8)        2048        bidirectional_38[0][0]           
___________________________________________________________________________________________

In [256]:
lstm_model_wrapper = Keras_Model_Wrapper(lstm_attention(), loss='binary_crossentropy')
cross_validation(lstm_model_wrapper, x_train, y_train)



Average Accuracy: 0.9125
Average F1:       0.47706
Average AUPRC:       0.0875
Average AUROC:       0.5
Average Accuracy per class
[1. 0.]




In [ ]:
lstm_model_wrapper = Keras_Model_Wrapper(lstm_attention(), loss='binary_crossentropy', epochs=100, batch_size=100)
cross_validation(lstm_model_wrapper, x_train, y_train)